<a href="https://colab.research.google.com/github/enoamoo/eno/blob/master/Amoo_Eno_stage_C_code.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Importing libraries

In [21]:
import numpy as np
import pandas as pd

In [22]:
#uploading data
csv_df=pd.read_csv('https://archive.ics.uci.edu/ml/machine-learning-databases/00471/Data_for_UCI_named.csv')
df = csv_df
df.describe(include='all')

,tau1,tau2,tau3,tau4,p1,p2,p3,p4,g1,g2,g3,g4,stab,stabf
count,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000.000000,10000
unique,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2
top,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,unstable
freq,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6380
mean,5.250000,5.250001,5.250004,5.249997,3.750000,-1.250000,-1.250000,-1.250000,0.525000,0.525000,0.525000,0.525000,0.015731,NaN
std,2.742548,2.742549,2.742549,2.742556,0.752160,0.433035,0.433035,0.433035,0.274256,0.274255,0.274255,0.274255,0.036919,NaN
min,0.500793,0.500141,0.500788,0.500473,1.582590,-1.999891,-1.999945,-1.999926,0.050009,0.050053,0.050054,0.050028,-0.080760,NaN
25%,2.874892,2.875140,2.875522,2.874950,3.218300,-1.624901,-1.625025,-1.624960,0.287521,0.287552,0.287514,0.287494,-0.015557,NaN
50%,5.250004,5.249981,5.249979,5.249734,3.751025,-1.249966,-1.249974,-1.250007,0.525009,0.525003,0.525015,0.525002,0.017142,NaN
75%,7.624690,7.624893,7.624948,7.624838,4.282420,-0.874977,-0.875043,-0.875065,0.762435,0.762490,0.762440,0.762433,0.044878,NaN


In [23]:
#check distribution of target variable
df['stabf'].value_counts()

unstable    6380
stable      3620
Name: stabf, dtype: int64

In [24]:
#dropping 'stab' column
df.drop(columns = 'stab', inplace=True)

In [25]:
#split the data into training and testing sets
X = df.drop(columns='stabf')
y = df['stabf']
from sklearn.model_selection import train_test_split
x_train, x_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)
y_train.value_counts()

unstable    5092
stable      2908
Name: stabf, dtype: int64

# Feature Scaling

In [26]:
#feature scaling 
from sklearn.preprocessing import StandardScaler
sc=StandardScaler()
x_train = sc.fit_transform(x_train)
x_test = sc.transform(x_test)

# RandomForest CLassifier

In [27]:
from sklearn.ensemble import RandomForestClassifier
clf = RandomForestClassifier(random_state=1)
clf.fit(x_train, y_train)

RandomForestClassifier(bootstrap=True, ccp_alpha=0.0, class_weight=None,
                       criterion='gini', max_depth=None, max_features='auto',
                       max_leaf_nodes=None, max_samples=None,
                       min_impurity_decrease=0.0, min_impurity_split=None,
                       min_samples_leaf=1, min_samples_split=2,
                       min_weight_fraction_leaf=0.0, n_estimators=100,
                       n_jobs=None, oob_score=False, random_state=1, verbose=0,
                       warm_start=False)

In [28]:
from sklearn.metrics import classification_report, recall_score, accuracy_score, precision_score, f1_score, confusion_matrix
predictions = clf.predict(x_test,)
print('RandomForestClassifier Report\n', classification_report(y_test, predictions, digits = 4))


RandomForestClassifier Report
               precision    recall  f1-score   support

      stable     0.9191    0.8778    0.8980       712
    unstable     0.9341    0.9573    0.9456      1288

    accuracy                         0.9290      2000
   macro avg     0.9266    0.9176    0.9218      2000
weighted avg     0.9288    0.9290    0.9286      2000



In [29]:
#prints confusion_matrix
print(confusion_matrix(y_test, predictions))

[[ 625   87]
 [  55 1233]]


In [30]:
#prints Accuracy
print(accuracy_score(y_test, predictions))

0.929


# ExtraTree Classifier

In [31]:
from sklearn.ensemble import ExtraTreesClassifier
xtree = ExtraTreesClassifier()
n_estimators = [50, 100, 300, 500, 1000]

min_samples_split = [2, 3, 5, 7, 9]

min_samples_leaf = [1, 2, 4, 6, 8]

max_features = ['auto', 'sqrt', 'log2', None] 

In [32]:
hyperparameter_grid = {'n_estimators': n_estimators,

                       'min_samples_leaf': min_samples_leaf,

                       'min_samples_split': min_samples_split,

                       'max_features': max_features}

In [33]:
from sklearn.model_selection import RandomizedSearchCV
clf = RandomizedSearchCV(xtree, hyperparameter_grid, random_state=1)
clf.fit(x_train, y_train)

RandomizedSearchCV(cv=None, error_score=nan,
                   estimator=ExtraTreesClassifier(bootstrap=False,
                                                  ccp_alpha=0.0,
                                                  class_weight=None,
                                                  criterion='gini',
                                                  max_depth=None,
                                                  max_features='auto',
                                                  max_leaf_nodes=None,
                                                  max_samples=None,
                                                  min_impurity_decrease=0.0,
                                                  min_impurity_split=None,
                                                  min_samples_leaf=1,
                                                  min_samples_split=2,
                                                  min_weight_fraction_leaf=0.0,
                                     

In [53]:
clf.best_params_

{'max_features': None,
 'min_samples_leaf': 8,
 'min_samples_split': 2,
 'n_estimators': 1000}

In [34]:
xtree_pred = clf.predict(x_test)

In [35]:
print(classification_report(y_test, xtree_pred, digits = 4))

              precision    recall  f1-score   support

      stable     0.9210    0.8680    0.8937       712
    unstable     0.9293    0.9589    0.9438      1288

    accuracy                         0.9265      2000
   macro avg     0.9251    0.9134    0.9188      2000
weighted avg     0.9263    0.9265    0.9260      2000



In [36]:
# Prints Accuracy
print(accuracy_score(y_test,xtree_pred))

0.9265


In [37]:
#Confusion_Matrix
print(confusion_matrix(y_test, xtree_pred))

[[ 618   94]
 [  53 1235]]


# XGBOOST and LIGHTGBM

In [38]:
pip install xgboost

In [39]:
pip install lightgbm

In [40]:
from lightgbm import LGBMClassifier

In [41]:
lgbm = LGBMClassifier(random_state=1)

In [42]:
lgbm.fit(x_train, y_train)

LGBMClassifier(boosting_type='gbdt', class_weight=None, colsample_bytree=1.0,
               importance_type='split', learning_rate=0.1, max_depth=-1,
               min_child_samples=20, min_child_weight=0.001, min_split_gain=0.0,
               n_estimators=100, n_jobs=-1, num_leaves=31, objective=None,
               random_state=1, reg_alpha=0.0, reg_lambda=0.0, silent=True,
               subsample=1.0, subsample_for_bin=200000, subsample_freq=0)

In [43]:
from xgboost import XGBClassifier

In [44]:
xgb = XGBClassifier(random_state=1)
xgb.fit(x_train, y_train)

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=1,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

In [45]:
xgb_pred = xgb.predict(x_test)

In [46]:
lgbm_pred = lgbm.predict(x_test)

In [47]:
print('classification Report of XGBoost\n', classification_report(y_test, xgb_pred, digits = 4))

classification Report of XGBoost
               precision    recall  f1-score   support

      stable     0.9206    0.8469    0.8822       712
    unstable     0.9190    0.9596    0.9389      1288

    accuracy                         0.9195      2000
   macro avg     0.9198    0.9033    0.9105      2000
weighted avg     0.9195    0.9195    0.9187      2000



In [48]:
print(confusion_matrix(y_test,xgb_pred))

[[ 603  109]
 [  52 1236]]


In [49]:
print(accuracy_score(y_test,xgb_pred))

0.9195


In [50]:
print('classification Report of Lightgbm\n', classification_report(y_test, lgbm_pred, digits = 4))

classification Report of Lightgbm
               precision    recall  f1-score   support

      stable     0.9297    0.8919    0.9104       712
    unstable     0.9415    0.9627    0.9520      1288

    accuracy                         0.9375      2000
   macro avg     0.9356    0.9273    0.9312      2000
weighted avg     0.9373    0.9375    0.9372      2000



In [51]:
print(confusion_matrix(y_test,lgbm_pred))

[[ 635   77]
 [  48 1240]]


In [52]:
print(accuracy_score(y_test,lgbm_pred))

0.9375
